In [1]:
# Import libriries
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from gensim.models import KeyedVectors
from time import time
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime
from utils import *
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
import pickle


Using TensorFlow backend.


In [9]:
filename = 'embeddings'
infile = open(filename,'rb')
embeddings = pickle.load(infile)
embedding_dim = 300

filename = 'df'
infile = open(filename,'rb')
df = pickle.load(infile)


In [10]:
train_df = df
questions_cols = ['enc_question1', 'enc_question2']
max_seq_length = max(train_df.enc_question1.map(lambda x: len(x)).max(),
                     train_df.enc_question2.map(lambda x: len(x)).max())
# Split to train validation
validation_size = 40000
training_size = len(train_df) - validation_size

X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.enc_question1, 'right': X_train.enc_question2}
X_validation = {'left': X_validation.enc_question1, 'right': X_validation.enc_question2}
# X_test = {'left': test_df.question1, 'right': test_df.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [ ]:
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 10


# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden, activation='tanh', recurrent_activation='hard_sigmoid')

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)


# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])
# coslstm_distance = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm) # other possible optimizers

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy']) #cross entropy, NLL, 
# Start training
training_start_time = time()

checkpointer = ModelCheckpoint(filepath='cosine/weights.{epoch:02d}-{val_loss:.4f}-{val_acc:.4f}.hdf5', verbose=1, save_best_only=True)

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch, 
                            callbacks=[checkpointer], validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))